# U-Net Model Notebook

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Import this for the relu function

class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        # Encoder
        self.e11 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.e12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e21 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.e22 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e31 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.e32 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e41 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.e42 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e51 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)

        # Decoder
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3, padding=1)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # Output layer
        self.outconv = nn.Conv2d(64, n_class, kernel_size=1)

    def forward(self, x):
        # Encoder
        xe11 = F.relu(self.e11(x))
        xe12 = F.relu(self.e12(xe11))
        xp1 = self.pool1(xe12)

        xe21 = F.relu(self.e21(xp1))
        xe22 = F.relu(self.e22(xe21))
        xp2 = self.pool2(xe22)

        xe31 = F.relu(self.e31(xp2))
        xe32 = F.relu(self.e32(xe31))
        xp3 = self.pool3(xe32)

        xe41 = F.relu(self.e41(xp3))
        xe42 = F.relu(self.e42(xe41))
        xp4 = self.pool4(xe42)

        xe51 = F.relu(self.e51(xp4))
        xe52 = F.relu(self.e52(xe51))

        # Decoder
        xu1 = self.upconv1(xe52)
        xu11 = torch.cat([xu1, xe42], dim=1)
        xd11 = F.relu(self.d11(xu11))
        xd12 = F.relu(self.d12(xd11))

        xu2 = self.upconv2(xd12)
        xu22 = torch.cat([xu2, xe32], dim=1)
        xd21 = F.relu(self.d21(xu22))
        xd22 = F.relu(self.d22(xd21))

        xu3 = self.upconv3(xd22)
        xu33 = torch.cat([xu3, xe22], dim=1)
        xd31 = F.relu(self.d31(xu33))
        xd32 = F.relu(self.d32(xd31))

        xu4 = self.upconv4(xd32)
        xu44 = torch.cat([xu4, xe12], dim=1)
        xd41 = F.relu(self.d41(xu44))
        xd42 = F.relu(self.d42(xd41))

        # Output layer
        out = self.outconv(xd42)

        return out


In [7]:
import torch
from Model import UNet  # Import your U-Net model from Model.py or Model.ipynb

# Instantiate your U-Net model
model = UNet(n_class=1)  # Adjust n_class according to your task (e.g., number of classes for segmentation)

# Set your model to evaluation mode
model.eval()

# Assuming images_tensor is already loaded and preprocessed
# It should be of shape (batch_size, channels, height, width)
# Example: images_tensor.shape => torch.Size([batch_size, 3, 256, 256])

# Run inference
with torch.no_grad():
    output = model(images_tensor)

# 'output' will contain the predictions; you can further process or use this output as needed
# Example: output.shape => torch.Size([batch_size, n_class, height, width])

# Example: Save or visualize the output (adjust as per your requirement)
# Assuming 'output' is in the format suitable for your task
# Example: output.shape => torch.Size([batch_size, n_class, height, width])



Epoch 0/59
----------
LR 0.0001
train: bce: 0.685497, dice: 0.990572, loss: 0.838035
val: bce: 0.674462, dice: 0.990246, loss: 0.832354
saving best model
2m 2s
Epoch 1/59
----------
LR 0.0001
train: bce: 0.668284, dice: 0.990540, loss: 0.829412
val: bce: 0.655759, dice: 0.990242, loss: 0.823001
saving best model
1m 60s
Epoch 2/59
----------
LR 0.0001
train: bce: 0.644974, dice: 0.990533, loss: 0.817754
val: bce: 0.623436, dice: 0.990253, loss: 0.806844
saving best model
2m 1s
Epoch 3/59
----------
LR 0.0001
train: bce: 0.603726, dice: 0.990557, loss: 0.797142
val: bce: 0.561696, dice: 0.990320, loss: 0.776008
saving best model
1m 59s
Epoch 4/59
----------
LR 0.0001
train: bce: 0.517221, dice: 0.990693, loss: 0.753957
val: bce: 0.411241, dice: 0.990858, loss: 0.701050
saving best model
1m 60s
Epoch 5/59
----------
LR 0.0001
train: bce: 0.279259, dice: 0.992659, loss: 0.635959
val: bce: 0.063730, dice: 0.997429, loss: 0.530580
saving best model
1m 48s
Epoch 6/59
----------
LR 0.0001
trai